In [1]:
TEXT2SOURCEDICT = {}

In [2]:
import re
import pandas as pd
import os
def depersonalize(text):
    text = str(text)
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text= re.sub(url_regex, "url", text)
    text = re.sub("id[\d]*",'',text)
    text = re.sub("> ?",'',text)
    text = re.sub("@[\w]*",'',text)
#     text = re.sub("\+[\d]*", "", text)
    text = re.sub("[\d]+", "NUMBER", text)
    text = re.sub(">>", "", text)
#     text = re.sub("[\d]{3,100}", "", text)
    text = re.sub("[\t|\n|\r]", "", text)
    text = re.sub("[(OP)|url|nickname|phone_number]", "", text)
    return text

In [3]:
# tr test

In [12]:
def get_texts_from_folder(folder, txt, sep):
    txt_set = set()
    for f in os.listdir(folder):
        pth = os.path.join(folder, f)
        df_t = pd.read_csv(pth, sep=sep)
        texts_set = set(df_t[txt].tolist())
        txt_set = txt_set.union(texts_set)
    return txt_set

In [5]:
train_tes_texts_set = get_texts_from_folder('./gr0_train/',"INPUT:text", '\t')

In [6]:
fpaths = ['g1.tsv','g2.tsv','g3.tsv','g1_4.tsv','g2_4.tsv','g3_4.tsv','maybe_old/g1.tsv','maybe_old/g2.tsv']
for f in fpaths:
    pth = './directly_from_toloka/' + f
    df_c = pd.read_csv(pth, sep = '\t')
    golden_columns = [c for c in df_c.columns if 'GOLDEN' in c]
    df_c = df_c[~df_c[golden_columns[1]].isna()]
    train_tes_texts_set = train_tes_texts_set.union(set(df_c['INPUT:text'].tolist()))

In [7]:
def add_set_to_dict(text_set, category):
    for t in text_set:
        if t not in TEXT2SOURCEDICT:
            TEXT2SOURCEDICT[t] = category
            
        t_pr = depersonalize(t)
        if t_pr not in TEXT2SOURCEDICT:
            TEXT2SOURCEDICT[t_pr] = category

In [8]:
add_set_to_dict(train_tes_texts_set, "trtest")

In [9]:
len(TEXT2SOURCEDICT)

912

In [ ]:
# toloka

In [10]:
toloka_set = get_texts_from_folder('./gr1_toloka/','text',',')

In [13]:
add_set_to_dict(toloka_set, "toloka")

In [14]:
len(TEXT2SOURCEDICT)

12939

In [15]:
# "self"

In [16]:
df1_s = pd.read_csv('./gr2_self/export_ready_st1 - Sheet1.csv')
df2_s = pd.read_csv('./gr2_self/SELF_LABEL_export_ready_st2_multilabel_view.csv')
df2_s = df2_s[:1000]
df_self = pd.concat([df1_s,df2_s])

In [17]:
self_list = set(df_self['text'].tolist())
len(self_list)

2009

In [18]:
add_set_to_dict(self_list, "self")
len(TEXT2SOURCEDICT)

15356

In [19]:
# kw

In [20]:
df2_s = pd.read_csv('./gr2_self/SELF_LABEL_export_ready_st2_multilabel_view.csv')
df2_s = df2_s[1000:]
df3_s = pd.read_csv('./gr2_self/SELF_LABLE_export_ready_st3_multilabel_view.csv')
df4_s = pd.read_csv('./gr2_self/SELF_LABLE_export_ready_st4_multilabel_view.csv')
df_kw = pd.concat([df2_s,df3_s,df4_s])
kw_list = set(df_kw['text'].tolist())

In [21]:
add_set_to_dict(kw_list, "kw")
len(TEXT2SOURCEDICT)

51849

In [22]:
# "kwolga"

In [23]:
df_kwolga_list = []
folder = './gr3_kw_olga/'
for f in os.listdir(folder):
    filepath = os.path.join(folder, f)
    df_curr = pd.read_csv(filepath, header = None)
    df_kwolga_list.append(df_curr)
    
df_kwolga = pd.concat(df_kwolga_list)

In [24]:
kwolga_set = set(df_kwolga[0].tolist())

In [25]:
add_set_to_dict(kwolga_set, "kwolga")
len(TEXT2SOURCEDICT)

51887

In [26]:
# Считаем

In [27]:
from collections import Counter

In [28]:
df_sensitive_published = pd.read_csv("../sensitive_topics/sensitive_topics.csv")

In [29]:
len(df_sensitive_published)

33303

In [30]:
source_list = []
not_found = 0
for t in df_sensitive_published['text'].tolist():
    t_proc = depersonalize(t)
    if t in TEXT2SOURCEDICT:
        source_list.append(TEXT2SOURCEDICT[t])
    elif t_proc in TEXT2SOURCEDICT:
        source_list.append(TEXT2SOURCEDICT[t_proc])
    else:
        not_found += 1
        source_list.append('toloka')
print(" {} marked as toloka but actually not found".format(not_found))

 365 marked as toloka but actually not found


In [36]:
Counter(source_list).most_common(100)

[('kw', 21656), ('toloka', 9291), ('self', 1978), ('trtest', 378)]

In [37]:
train_tes_texts_set_proc = set([depersonalize(t) for t in train_tes_texts_set])

In [38]:
Counter(df_sensitive_published['text'].isin(train_tes_texts_set_proc))

Counter({False: 32925, True: 378})

In [39]:
df_sensitive_published['source'] = source_list

In [40]:
df_sensitive_published.to_csv("sensitive_topics_vs_source.csv", index = None)